# Klassifikation von Rot- und Weißwein mit Keras
In diesem Notebook führen wir die vollständige Datenaufbereitung, Modellierung, Visualisierung und Evaluierung eines neuronalen Netzes zur Unterscheidung von Rot- und Weißwein durch.

In [ ]:
import pandas as pd
import numpy as np
import keras as K
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Einlesen der Weindaten
Die CSV-Dateien befinden sich im Unterordner `data/processed`. Sie enthalten standardisierte chemische Merkmale für Rot- und Weißweine.

In [ ]:
white = pd.read_csv('../data/processed/winequality-white.csv', sep=';')
red = pd.read_csv('../data/processed/winequality-red.csv', sep=';')

## Labeln und Zusammenführen der Datensätze
Rotwein erhält das Label `1`, Weißwein das Label `0`. Danach werden die Daten zusammengeführt.

In [ ]:
red['label'] = 1
white['label'] = 0
wines = pd.concat([red, white], ignore_index=True)

## Feature-Matrix und Zielvariable erzeugen
Die ersten 11 Spalten (chemische Eigenschaften) dienen als Input-Variablen (Features). Die Zielvariable `y` ist das binäre Label.

In [ ]:
x = wines.iloc[:, 0:11]
y = np.ravel(wines['label'])

## Aufteilung in Trainings- und Testdaten
Der Datensatz wird im Verhältnis 70:30 in Trainings- und Testdaten aufgeteilt.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42)

## Skalierung der Merkmale
Die Daten werden mit dem `StandardScaler` auf Mittelwert 0 und Standardabweichung 1 standardisiert. Der Fit erfolgt **nur auf dem Trainingsset**.

In [ ]:
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

## Aufbau des neuronalen Netzes
Wir definieren ein sequentielles Keras-Modell mit folgender Struktur:
- Eingabeschicht: 12 Neuronen, ReLU, `input_dim=11`
- Hidden Layer: 8 Neuronen, ReLU
- Ausgabeschicht: 1 Neuron, Sigmoid (für binäre Klassifikation)

In [ ]:
model = K.models.Sequential()
model.add(K.layers.Dense(units=12, activation='relu', input_dim=11))
model.add(K.layers.Dense(units=8, activation='relu'))
model.add(K.layers.Dense(units=1, activation='sigmoid'))

## Kompilierung des Modells
- Optimierer: Adam
- Verlustfunktion: `binary_crossentropy`
- Metrik: `accuracy`

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

## Modelltraining
Wir trainieren das Modell 20 Epochen lang mit `validation_split=0.3`.

In [ ]:
hist = model.fit(x_train, y_train, epochs=20, validation_split=0.3)

## Verlauf der Genauigkeit während des Trainings
Die Trainings- und Validierungsgenauigkeit werden für jede Epoche geplottet.

In [ ]:
plt.plot(hist.history['accuracy'], label='Train Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoche')
plt.ylabel('Genauigkeit')
plt.legend()
plt.tight_layout()
plt.savefig('../figs/training_accuracy.png')
plt.show()

## Evaluation auf Testdaten
Nach dem Training evaluieren wir das Modell auf der echten Testmenge und geben die Loss- und Accuracy-Werte aus.

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Testverlust: {test_loss:.4f}")
print(f"Testgenauigkeit: {test_acc:.4f}")

**Hinweis:**
- Die Validierung während des Trainings basiert auf `validation_split`
- Die finale Testgenauigkeit stammt aus einer separaten, vorher unberührten Testmenge
- Dadurch erhalten wir eine realistische Einschätzung der Generalisierungsfähigkeit